# **Convulotional Neural Networks**

The followings code are inspired from
"Image Preparation for Convolutional Neural Networks with TensorFlow's Keras API" video by deeplizard

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Model
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings

warnings.simplefilter (action='ignore', category=FutureWarning)
%matplotlib inline

print(np.__version__)

In [ ]:
# Check for available GPUs and set memory growth for the first GPU

# List all available physical devices (including GPUs)
physical_devices = tf.config.experimental.list_physical_devices('GPU')

# Print the number of available GPUs
print("Num GPUs Available: ", len(physical_devices))

# Set memory growth for the first GPU (assuming at least one GPU is available)
tf.config.experimental.set_memory_growth(physical_devices[0], True)


# **Data preparation**

In [ ]:
# Change the current working directory to '/kaggle/working'
os.chdir('/kaggle/working')

# Define the source directory where the dataset is located
source_directory = '/kaggle/input/laguna-banana-dataset-preprocessed/laguna_dataset_preprocessed'

# # Define the path to the directory you want to remove
# directory_to_remove = 'laguna_dataset_preprocessed'
#  # If it exists, remove it
# shutil.rmtree(directory_to_remove)
    
# Check if a directory named 'dogcat_dataset' does not exist
if os.path.isdir('laguna_dataset_preprocessed') is False:
    # If 'dogcat_dataset' does not exist, copy the contents of 'source_directory' to 'dogcat_dataset'
    shutil.copytree(source_directory, 'laguna_dataset_preprocessed')


In [ ]:
# Define paths for training, validation, and test sets
train_path = '/kaggle/working/laguna_dataset_preprocessed/train'
valid_path = '/kaggle/working/laguna_dataset_preprocessed/valid'
test_path = '/kaggle/working/laguna_dataset_preprocessed/test'

# Check if the directories exist and print the results
print(os.path.isdir(train_path))  # Check if 'train' directory exists
print(os.path.isdir(valid_path))  # Check if 'valid' directory exists
print(os.path.isdir(test_path))   # Check if 'test' directory exists


In [ ]:
# Define an ImageDataGenerator for data augmentation and preprocessing using MobileNet settings
train_datagen = ImageDataGenerator(
    rotation_range=20,  # Increased rotation range
    width_shift_range=0.2,  # Increased shift range
    height_shift_range=0.2,  # Increased shift range
    shear_range=0.2,  # Increased shear range
    zoom_range=0.2,  # Increased zoom range
    channel_shift_range=20.,  # Increased channel shift range
    horizontal_flip=True,
    preprocessing_function=tf.keras.applications.mobilenet.preprocess_input
)


In [ ]:
mobile = tf.keras.applications.mobilenet.MobileNet()

In [ ]:

# Define ImageDataGenerator and generate batches of images
# Apply data augmentation to the training batch
train_batches = train_datagen.flow_from_directory(
    directory=train_path,
    target_size=(224, 224), classes=['Black sigatoka', 'Bunchy top', 'Healthy'], batch_size=10)

# Validation set
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=valid_path, target_size=(224,224), classes=['Black sigatoka', 'Bunchy top', 'Healthy'], batch_size=10)

# Test set
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=test_path, target_size=(224,224), classes=['Black sigatoka', 'Bunchy top', 'Healthy'], batch_size=10, shuffle=False)


In [ ]:
# Assertions to validate properties of the generated batches

# Ensure that there are 2000 images in the training set
assert train_batches.n == 627

# Ensure that there are 400 images in the validation set
assert valid_batches.n == 177

# Ensure that there are 200 images in the test set
assert test_batches.n == 79

# Ensure that the number of classes is 3 for all sets (Black sigatoka, Bunchy top, Healthy)
assert train_batches.num_classes == valid_batches.num_classes == test_batches.num_classes == 3


In [ ]:
# Generate a batch of images and their corresponding labels

# Use the 'next' method to get the next batch of images and labels from the training set
imgs, labels = next(train_batches)
# Print out the labels
print(labels)

# # One-hot encode the labels
# from tensorflow.keras.utils import to_categorical
# one_hot_labels = to_categorical(labels, num_classes=3)

# # Print out the one-hot encoded labels
# print(one_hot_labels)

In [ ]:
# Define a function to plot a series of images

def plotImages(images_arr):
    # Create a figure with 10 subplots arranged in 1 row and 10 columns
    fig, axes = plt.subplots(1, 3, figsize=(20,20))
    
    # Flatten the array of axes to make it easier to iterate over
    axes = axes.flatten()
    
    # Iterate through the images and corresponding axes
    for img, ax in zip(images_arr, axes):
        # Display the image on the current axis
        ax.imshow(img)
        
        # Turn off axis labels for better visualization
        ax.axis('off')
    
    # Adjust the layout to prevent overlapping
    plt.tight_layout()
    
    # Display the figure with the images
    plt.show()


In [ ]:
# Plot the images in 'imgs' and print the corresponding labels

# Call the previously defined function 'plotImages' to display the images
plotImages(imgs)

# Print the labels corresponding to the displayed images
print(labels)


# **Build a Fine-Tuned MobileNet model**

The following codes are inspired from "Build a Fine-Tuned Neural Network with TensorFlow's Keras API" video by deeplizard

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical


In [ ]:
mobile = tf.keras.applications.mobilenet.MobileNet()

In [ ]:
mobile.summary()

In [ ]:
def count_params(model):
    non_trainable_params = np.sum([np.prod(v.get_shape().as_list()) for v in model.non_trainable_weights])
    trainable_params = np.sum([np.prod(v.get_shape().as_list()) for v in model.trainable_weights])
    return {'non_trainable_params': non_trainable_params, 'trainable_params': trainable_params}

In [ ]:
params = count_params(mobile)
assert params['non_trainable_params'] == 21888
assert params['trainable_params'] == 4231976

In [ ]:
x = mobile.layers[-5].output

x = tf.keras.layers.Reshape(target_shape=(1024,))(x)

x = Dropout(0.8)(x)  # Add dropout layer

output = Dense(units=3, activation='softmax', 
               kernel_regularizer=regularizers.l1_l2(l1=0.1, l2=0.1))(x)  # ElasticNet Regularization

In [ ]:
model = Model(inputs=mobile.input, outputs=output)

In [ ]:
for layer in model.layers[:-50]:
    layer.trainable = False

In [ ]:
model.summary()

In [ ]:
# params = count_params (model)
# assert params['non_trainable_params'] == 1371840
# assert params['trainable_params'] == 1860099

# **Train the Fine-Tuned MobileNet model**

The following codes are inspired from "Train a Fine-Tuned Neural Network with TensorFlow's Keras API" video by deeplizard

In [ ]:
num_Bunchy_top_samples = len(os.listdir('/kaggle/working/laguna_dataset_preprocessed/test/Bunchy top'))
num_Black_sigatoka_samples = len(os.listdir('/kaggle/working/laguna_dataset_preprocessed/test/Black sigatoka'))
num_Healthy_samples = len(os.listdir('/kaggle/working/laguna_dataset_preprocessed/test/Healthy'))

print("Number of Bunchy top samples:", num_Bunchy_top_samples)
print("Number of Black sigatoka samples:", num_Black_sigatoka_samples)
print("Number of Healthy samples:", num_Healthy_samples)

In [ ]:
# early_stopping = EarlyStopping(monitor='val_loss', patience=7)  # Adjust patience as needed

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=1000,
    decay_rate=0.9)
optimizer = Adam(learning_rate=lr_schedule)

In [ ]:
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(x=train_batches,
            steps_per_epoch=len(train_batches),
            validation_data=valid_batches,
            validation_steps=len(valid_batches),
            epochs=20,
            verbose=2,
            callbacks=[early_stopping]
)

In [ ]:
# Retrieve training and validation accuracy from the training history
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Create a list of epochs for the x-axis
epochs = range(1, len(train_accuracy) + 1)

# Plot the training and validation accuracy
plt.plot(epochs, train_accuracy, 'b', label='Training Accuracy')  # Blue line for training accuracy
plt.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')  # Red line for validation accuracy

# Add title and labels to the plot
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

# Add a legend to differentiate between training and validation accuracy
plt.legend()

# Show the plot
plt.show()

In [ ]:
# Retrieve training and validation accuracy from the training history
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Create a list of epochs for the x-axis
epochs = range(1, len(train_loss) + 1)

# Plot the training and validation accuracy
plt.plot(epochs, train_loss, 'b', label='Training Loss')  # Blue line for training accuracy
plt.plot(epochs, val_loss, 'r', label='Validation Loss')  # Red line for validation accuracy

# Add title and labels to the plot
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

# Add a legend to differentiate between training and validation accuracy
plt.legend()

# Show the plot
plt.show()

In [ ]:
# assert model.history.history.get('accuracy')[-1] > 0.80

# **Predict using Fine-Tuned MobileNet model**

The following codes are inspired from "Predict with a Fine-Tuned Neural Network with TensorFlow's Keras API" video by deeplizard

In [ ]:
predictions = model.predict(x=test_batches, verbose=0)

In [ ]:
test_batches.classes

In [ ]:
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting 'normalize=True'.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
        
    print(cm)
    
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape [0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black")
        
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
test_batches.class_indices

In [ ]:
# Define class labels for plotting the confusion matrix
cm_plot_labels = ['Black sigatoka', 'Bunchy top', 'Healthy']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Compute the confusion matrix using ground truth classes and predicted class labels 
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))

# Calculate the accuracy 
accuracy = accuracy_score(test_batches.classes, np.argmax(predictions, axis=1))

# Convert the accuracy to a percentage 
accuracy_percentage = accuracy * 100.0

# Print the test accuracy and confusion matrix 
print("Test Accuracy: {:.2f}%".format(accuracy_percentage))
print("Confusion Matrix:")
print(cm)


In [ ]:
# Get the class labels for the images in the test set
test_classes = test_batches.classes

# Make predictions on the test set
predictions = model.predict(x=test_batches, verbose=0)

# Convert the predicted probabilities to class labels
predicted_classes = np.argmax(predictions, axis=-1)

# Get the filenames of the test images
filenames = test_batches.filenames

# Print out images along with their true and predicted labels
for i in range(len(filenames)):
    print(f"True Label: {test_classes[i]}, Predicted Label: {predicted_classes[i]}, File Name: {filenames[i]}")


# **Saving the Model**

In [ ]:
model.summary()

In [ ]:
try:
    model.save('Laguna_Banana_Model.h5')
    print("Model saved successfully.")
except Exception as e:
    print(f"An error occurred while saving the model: {e}")


In [ ]:
from tensorflow.keras.models import load_model
new_model = load_model('/kaggle/working/Laguna_Banana_Model.h5')

In [ ]:
new_model.summary()

In [ ]:
new_model.get_weights()

In [ ]:
new_model.optimizer